In [131]:
import pykx as kx
import pandas as pd
import numpy as np
import plotly.express as px
# import matplotlib.pyplot as plt
import datetime
import pytz


## define connection to Gateway
gw = kx.QConnection(host='localhost', port=49164, no_ctx = True)

END_TIME = datetime.datetime.now(tz=pytz.utc)                                       ## Now
START_TIME = END_TIME - datetime.timedelta(minutes = 300)                           ## 300 Mins ago

## Query 1
query_params = {
    'table': 'trade',
    'startTS': START_TIME,
    'endTS': END_TIME,
    'sortCols': 'time',
    'filter': [[b'in','sym','IBM']]
}

empty_dict = {'':''}

trade = gw(kx.SymbolAtom('.kxi.getData'), query_params, 'f', empty_dict)[1]

## Query 2
query_params = {
    'table': 'quote',
    'startTS': START_TIME,
    'endTS': END_TIME,
    'sortCols': 'time',
    'filter': [[b'in','sym','IBM']]
}

quote = gw(kx.SymbolAtom('.kxi.getData'), query_params, 'f', empty_dict)[1]

df = kx.q.aj(np.array(['sym', 'time']), trade, quote).pd()

# px.line(df, x='time', y=['price','bid','ask']).show() 
# px.line(df, x='time', y='price').show() 

# kx.q('select High:max price, Low:min price, Avg:avg price by 10 xbar time.minute from $1', df)


# res = kx.q.qsql.select(
#                 table = df, 
#                 columns = {'High':'max price', 'Low':'min price', 'Avg':'avg price'},
#                 by = {'T': ['.q.xbar', 360000, 'time']}
#                 )

# t = res.pd().rename_axis('Time')

# print(t.index)



# px.line(df, x='Time', y=['High','Low','Avg']).show() 
px.line(df, x='time', y='price').show() 
######### IAN #########
# containerize jupyter notebook 

# open, high, low, close

# xbar in ANSI SQL - not available until next week
# kx.q.sql('select xbar(10,price) from $1', trade)



######### JACK #########
# trade impact report
# spread before and after a trade
